In [46]:
import pandas as pd
import random 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm

import researchpy as rp
import scipy.stats as stats
from scipy.stats import levene
from scipy.stats import kstest
from scipy.stats import skew

import math 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA


import warnings
warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"] = (20,10) 
pd.options.display.max_columns = None

##### <font color=green> En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.

In [47]:
df= pd.read_pickle("datos/cars_price_norm_est.pkl")
df.head(2) 

,Levy,Manufacturer,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Color,Airbags,Price_raiz
0,1.766963,LEXUS,-0.411248,Jeep,Yes,Hybrid,3.5,0.237134,1.375313,Automatic,4x4,04,Silver,1.329406,115.446958
1,0.943936,CHEVROLET,-0.173337,Jeep,No,Petrol,3,0.273438,1.375313,Tiptronic,4x4,04,Black,0.319668,128.922457


In [48]:
df.dtypes

Levy                 float64
Manufacturer          object
Prod. year           float64
Category              object
Leather interior      object
Fuel type             object
Engine volume         object
Mileage              float64
Cylinders            float64
Gear box type         object
Drive wheels          object
Doors               category
Color                 object
Airbags              float64
Price_raiz           float64
dtype: object

##### <font color=green> Hasta ahora habéis estado evaluando las características de vuestro set de datos y habéis hecho una gran exploración, es el momento de hacer vuestro primer ANOVA! En el ejercicio de hoy tendréis que hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.

##### <font color=green> 📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer cuando nos encontramos en esta situación.

Hemos detectado que para hacer el anova necesitamos que no haya espacios entre las palabras de los nombres de las columnas, al llegar a este punto, nos damos cuenta que es algo que ya tendriamos que haber modficado al hacer la primera limpieza... :( perooo, más vale tarde que nunca :) 

In [49]:
df.columns 

Index(['Levy', 'Manufacturer', 'Prod. year', 'Category', 'Leather interior',
       'Fuel type', 'Engine volume', 'Mileage', 'Cylinders', 'Gear box type',
       'Drive wheels', 'Doors', 'Color', 'Airbags', 'Price_raiz'],
      dtype='object')

In [50]:
df.rename(columns={'Prod. year':'Prod_year', 'Leather interior':'Leather_interior', 'Fuel type':'Fuel_type', 'Engine volume':'Engine_volume', 'Gear box type':'Gear_box_type', 'Drive wheels':'Drive_wheels'}, inplace=True)

In [51]:
df.describe()

,Levy,Prod_year,Mileage,Cylinders,Airbags,Price_raiz
count,9877.000000,9877.000000,9877.000000,9877.000000,9877.000000,9877.000000
mean,0.007800,0.007608,0.001800,0.002635,0.000912,123.842703
std,1.002296,1.000678,1.004885,1.000446,1.000478,42.851586
min,-1.255121,-2.552451,-0.889269,-3.090707,-1.699806,23.430749
25%,-1.255121,-0.411248,-0.477477,-0.411095,-0.690069,98.600203
50%,0.127391,0.064574,-0.168633,-0.411095,-0.185200,127.702780
75%,0.645832,0.778309,0.164355,-0.411095,1.329406,148.165448
max,3.391414,1.967865,5.287479,10.307355,2.339143,217.041471


In [52]:
# iniciamos nuestro anova. Recordemos que la variable respuesta es el precio de los coches
lm = ols('Price_raiz ~ Levy + Manufacturer + Prod_year + Category + Leather_interior + Fuel_type + Engine_volume + Mileage + Cylinders + Gear_box_type + Drive_wheels + Doors + Color + Airbags', data = df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
Manufacturer,56.0,1.976499e+06,35294.623197,26.589375,1.324749e-253
Category,10.0,8.652256e+05,86522.560263,65.182190,3.852121e-129
Leather_interior,1.0,3.384553e+04,33845.525987,25.497691,4.509829e-07
Fuel_type,6.0,3.816872e+05,63614.528171,47.924313,2.916349e-58
Engine_volume,97.0,1.188171e+06,12249.181874,9.227981,1.222682e-122
Gear_box_type,3.0,6.747853e+05,224928.440984,169.450931,4.552862e-107
Drive_wheels,2.0,3.652755e+04,18263.776498,13.759105,1.078883e-06
Doors,2.0,1.146349e+04,5731.744392,4.318037,1.335169e-02
Color,15.0,1.132305e+05,7548.696795,5.686847,8.462611e-12
Levy,1.0,1.768690e+03,1768.689882,1.332451,2.483981e-01


In [53]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Price_raiz   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.277
Method:                 Least Squares   F-statistic:                     20.32
Date:                Wed, 05 Oct 2022   Prob (F-statistic):               0.00
Time:                        20:41:37   Log-Likelihood:                -49428.
No. Observations:                9877   AIC:                         9.925e+04
Df Residuals:                    9680   BIC:                         1.007e+05
Df Model:                         196                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        99.4452     24.148      4.118      0.000      52.109     146.781
Manufacturer[T.ALFA ROMEO]       -5.3688     22.555     -0.238      0.812     -49.581      38.844
Manufacturer[T.ASTON MARTIN]      6.8741     21.080      0.326      0.744     -34.446      48.194
Manufacturer[T.AUDI]              8.5765     13.587      0.631      0.528     -18.058      35.211
Manufacturer[T.BENTLEY]          24.7733     46.616      0.531      0.595     -66.605     116.151
Manufacturer[T.BMW]               9.7009     13.348      0.727      0.467     -16.464      35.866
Manufacturer[T.BUICK]            -6.1214     18.434     -0.332      0.740     -42.257      30.014
Manufacturer[T.CADILLAC]          2.9593     18.171      0.163      0.871     -32.660      38.578
Manufacturer[T.CHEVROLET]        15.1131     13.232      1.142      0.253     -10.825      41.051
Manufacturer[T.CHRYSLER]         -0.3292     15.676     -0.021      0.983     -31.058      30.400
Manufacturer[T.CITROEN]          -0.0363     18.537     -0.002      0.998     -36.372      36.300
Manufacturer[T.DAEWOO]           -8.5581     14.319     -0.598      0.550     -36.625      19.509
Manufacturer[T.DAIHATSU]        -18.4034     20.190     -0.912      0.362     -57.979      21.173
Manufacturer[T.DODGE]           -13.9952     14.098     -0.993      0.321     -41.630      13.640
Manufacturer[T.FERRARI]          61.9998     34.071      1.820      0.069      -4.786     128.786
Manufacturer[T.FIAT]             16.5097     14.443      1.143      0.253     -11.802      44.822
Manufacturer[T.FORD]             11.8518     13.221      0.896      0.370     -14.064      37.768
Manufacturer[T.GAZ]             -11.9016     38.877     -0.306      0.760     -88.109      64.306
Manufacturer[T.GMC]              -3.7509     18.045     -0.208      0.835     -39.123      31.621
Manufacturer[T.HAVAL]            -4.1976     39.277     -0.107      0.915     -81.189      72.794
Manufacturer[T.HONDA]            29.0826     13.270      2.192      0.028       3.071      55.094
Manufacturer[T.HUMMER]           42.8037     26.445      1.619      0.106      -9.034      94.641
Manufacturer[T.HYUNDAI]          34.7854     13.166      2.642      0.008       8.977      60.593
Manufacturer[T.INFINITI]         26.0392     16.182      1.609      0.108      -5.681      57.760
Manufacturer[T.JAGUAR]           18.6985     15.256      1.226      0.220     -11.206      48.603
Manufacturer[T.JEEP]             37.9451     13.630      2.784      0.005      11.228      64.662
Manufacturer[T.KIA]              24.3300     13.319      1.827      0.068      -1.778      50.438
Manufacturer[T.LANCIA]            1.9527     38.980      0.050      0.960     -74.457      78.363
Manufacturer[T.LAND ROVER]        2.7161     

#### <font color=violet> Análisis de Summary

Luego de analizar el p-valor para cada una de las variables catgóricas, observamos que hay algunas que no afectan nuestra variable `Precio`, es decir, hemos sacado el promedio del p-valor de estas y nos ha dado mayor a 0.05 (`p-valor>0.05 aceptamos nuestra hipótesis nula`). Por esta razón hemos decidido eliminar las variables `Manufacturer`, `Engine_Volumen`, `Color`, `Doors` y `Category` para continuar con nuestro proceso de encoding.

In [54]:
df.drop(["Manufacturer", "Engine_volume", "Color", "Doors", "Category"], axis = 1, inplace = True)
df.head(2)

,Levy,Prod_year,Leather_interior,Fuel_type,Mileage,Cylinders,Gear_box_type,Drive_wheels,Airbags,Price_raiz
0,1.766963,-0.411248,Yes,Hybrid,0.237134,1.375313,Automatic,4x4,1.329406,115.446958
1,0.943936,-0.173337,No,Petrol,0.273438,1.375313,Tiptronic,4x4,0.319668,128.922457


In [55]:
df.to_pickle('datos/cars_price_norm_est_an.pkl')

In [56]:
df.to_csv('datos/cars_price_norm_est_an.csv')
